In [ ]:
from typing import Dict, Any, List, TypedDict
from langgraph.graph import StateGraph, END
import re
import ast
try:
    from IPython.display import Image, display
    import matplotlib.pyplot as plt
    import matplotlib.patches as mpatches
    from matplotlib.patches import FancyBboxPatch
    import numpy as np
    VISUALIZATION_AVAILABLE = True
except ImportError:
    VISUALIZATION_AVAILABLE = False
    print("시각화 라이브러리가 없습니다. 그래프 시각화는 건너뜁니다.")

class AnalysisState(TypedDict):
    """분석 상태를 관리하는 TypedDict"""
    original_code: str
    style_analysis: Dict[str, Any]
    bug_analysis: Dict[str, Any]
    optimization_analysis: Dict[str, Any]
    improved_code: str
    analysis_complete: bool

class CodeAnalyzer:
    """코드 분석 및 개선을 위한 메인 클래스"""
    
    def __init__(self):
        self.graph = self._build_graph()
    
    def _build_graph(self):
        """LangGraph 워크플로우 구성"""
        workflow = StateGraph(AnalysisState)
        
        # 노드 추가
        workflow.add_node("style_check", self.check_style)
        workflow.add_node("bug_check", self.check_bugs)
        workflow.add_node("optimization_check", self.check_optimization)
        workflow.add_node("improve_code", self.improve_code)
        
        # 엣지 설정
        workflow.set_entry_point("style_check")
        workflow.add_edge("style_check", "bug_check")
        workflow.add_edge("bug_check", "optimization_check")
        workflow.add_edge("optimization_check", "improve_code")
        workflow.add_edge("improve_code", END)
        
        return workflow.compile()
    
    def check_style(self, state: AnalysisState) -> AnalysisState:
        """코드 스타일 분석 노드"""
        code = state["original_code"]
        issues = []
        suggestions = []
        
        # PEP 8 기본 체크
        lines = code.split('\n')
        
        for i, line in enumerate(lines, 1):
            # 라인 길이 체크
            if len(line) > 79:
                issues.append(f"Line {i}: 라인이 79자를 초과합니다 ({len(line)}자)")
                
            # 들여쓰기 체크 (4칸 공백)
            if line.startswith('\t'):
                issues.append(f"Line {i}: 탭 대신 4칸 공백을 사용하세요")
                
            # 함수/클래스명 체크
            if re.search(r'def [A-Z]', line):
                issues.append(f"Line {i}: 함수명은 snake_case를 사용하세요")
            if re.search(r'class [a-z]', line):
                issues.append(f"Line {i}: 클래스명은 PascalCase를 사용하세요")
        
        # import 순서 체크
        import_lines = [line for line in lines if line.strip().startswith(('import ', 'from '))]
        if import_lines:
            suggestions.append("import 문을 표준 라이브러리, 서드파티, 로컬 순서로 정렬하세요")
        
        # 공백 라인 체크
        if not re.search(r'\n\n(class |def )', code):
            suggestions.append("클래스와 함수 앞에 2줄의 공백을 추가하세요")
        
        return {
            **state,
            "style_analysis": {
                'issues': issues,
                'suggestions': suggestions,
                'score': max(0, 100 - len(issues) * 10)
            }
        }
    
    def check_bugs(self, state: AnalysisState) -> AnalysisState:
        """버그 가능성 분석 노드"""
        code = state["original_code"]
        potential_bugs = []
        warnings = []
        
        try:
            # AST 파싱으로 구문 오류 체크
            tree = ast.parse(code)
            
            # 일반적인 버그 패턴 체크
            lines = code.split('\n')
            
            for i, line in enumerate(lines, 1):
                # 나누기 0 가능성
                if '/' in line and not '//' in line:
                    if re.search(r'/\s*0\b', line):
                        potential_bugs.append(f"Line {i}: 0으로 나누기 오류 가능성")
                
                # 리스트 인덱스 오류 가능성
                if re.search(r'\[\s*-?\d+\s*\]', line):
                    potential_bugs.append(f"Line {i}: 인덱스 범위 오류 가능성 - 예외처리 고려")
                
                # 파일 처리시 close() 누락
                if 'open(' in line and 'with ' not in line:
                    warnings.append(f"Line {i}: 'with' 문을 사용한 안전한 파일 처리 권장")
                
                # except 문에서 구체적 예외 미지정
                if re.match(r'^\s*except\s*:', line):
                    warnings.append(f"Line {i}: 구체적인 예외 타입 지정 권장")
        
        except SyntaxError as e:
            potential_bugs.append(f"구문 오류: {str(e)}")
        
        return {
            **state,
            "bug_analysis": {
                'potential_bugs': potential_bugs,
                'warnings': warnings,
                'score': max(0, 100 - len(potential_bugs) * 20 - len(warnings) * 10)
            }
        }
    
    def check_optimization(self, state: AnalysisState) -> AnalysisState:
        """최적화 분석 노드"""
        code = state["original_code"]
        optimizations = []
        performance_tips = []
        
        lines = code.split('\n')
        
        for i, line in enumerate(lines, 1):
            # 리스트 컴프리헨션 사용 가능성
            if 'for ' in line and 'append(' in line:
                optimizations.append(f"Line {i}: 리스트 컴프리헨션 사용 고려")
            
            # 문자열 연결 최적화
            if '+=' in line and 'str' in line.lower():
                optimizations.append(f"Line {i}: join() 메서드나 f-string 사용 권장")
            
            # 반복문에서 len() 사용
            if re.search(r'for\s+\w+\s+in\s+range\s*\(\s*len\s*\(', line):
                optimizations.append(f"Line {i}: enumerate() 사용 권장")
            
            # 딕셔너리 키 체크
            if re.search(r'if\s+\w+\s+in\s+\w+\.keys\(\)', line):
                optimizations.append(f"Line {i}: '.keys()' 없이 'in dict' 사용 권장")
        
        # 전체 코드 패턴 분석
        if 'import *' in code:
            performance_tips.append("'import *' 대신 필요한 모듈만 import하세요")
        
        if code.count('print(') > 5:
            performance_tips.append("과도한 print문은 성능에 영향을 줄 수 있습니다")
        
        return {
            **state,
            "optimization_analysis": {
                'optimizations': optimizations,
                'performance_tips': performance_tips,
                'score': max(0, 100 - len(optimizations) * 15 - len(performance_tips) * 5)
            }
        }
    
    def improve_code(self, state: AnalysisState) -> AnalysisState:
        """코드 개선 노드"""
        code = state["original_code"]
        improved_code = code
        
        # 스타일 개선
        if state.get("style_analysis"):
            # 탭을 공백으로 변환
            improved_code = improved_code.replace('\t', '    ')
        
        # 버그 수정
        if state.get("bug_analysis"):
            # except 문 개선
            improved_code = re.sub(
                r'except\s*:',
                r'except Exception as e:',
                improved_code
            )
        
        return {
            **state,
            "improved_code": improved_code,
            "analysis_complete": True
        }
    
    def visualize_graph(self):
        """LangGraph 워크플로우 시각화"""
        if not VISUALIZATION_AVAILABLE:
            print("그래프 시각화를 위해 matplotlib가 필요합니다.")
            return
            
        fig, ax = plt.subplots(1, 1, figsize=(12, 8))
        
        # 노드 위치 정의
        nodes = {
            'START': (1, 4),
            'style_check': (3, 4),
            'bug_check': (5, 4),
            'optimization_check': (7, 4),
            'improve_code': (9, 4),
            'END': (11, 4)
        }
        
        # 노드 그리기
        for node_name, (x, y) in nodes.items():
            if node_name in ['START', 'END']:
                # 시작/끝 노드는 원형
                circle = plt.Circle((x, y), 0.3, color='lightgreen' if node_name == 'START' else 'lightcoral', 
                                  ec='black', linewidth=2)
                ax.add_patch(circle)
                ax.text(x, y, node_name, ha='center', va='center', fontweight='bold', fontsize=10)
            else:
                # 처리 노드는 사각형
                rect = FancyBboxPatch((x-0.6, y-0.3), 1.2, 0.6, 
                                    boxstyle="round,pad=0.1", 
                                    facecolor='lightblue', 
                                    edgecolor='black', 
                                    linewidth=2)
                ax.add_patch(rect)
                # 노드 이름을 두 줄로 표시
                if node_name == 'style_check':
                    ax.text(x, y, 'Style\nCheck', ha='center', va='center', fontweight='bold', fontsize=9)
                elif node_name == 'bug_check':
                    ax.text(x, y, 'Bug\nCheck', ha='center', va='center', fontweight='bold', fontsize=9)
                elif node_name == 'optimization_check':
                    ax.text(x, y, 'Optimization\nCheck', ha='center', va='center', fontweight='bold', fontsize=9)
                elif node_name == 'improve_code':
                    ax.text(x, y, 'Improve\nCode', ha='center', va='center', fontweight='bold', fontsize=9)
        
        # 화살표 그리기
        arrows = [
            ('START', 'style_check'),
            ('style_check', 'bug_check'),
            ('bug_check', 'optimization_check'),
            ('optimization_check', 'improve_code'),
            ('improve_code', 'END')
        ]
        
        for start, end in arrows:
            start_pos = nodes[start]
            end_pos = nodes[end]
            
            # 화살표 시작과 끝 점 조정 (노드 경계에서 시작/끝)
            if start == 'START':
                start_x = start_pos[0] + 0.3
            else:
                start_x = start_pos[0] + 0.6
            start_y = start_pos[1]
            
            if end == 'END':
                end_x = end_pos[0] - 0.3
            else:
                end_x = end_pos[0] - 0.6
            end_y = end_pos[1]
            
            ax.annotate('', xy=(end_x, end_y), xytext=(start_x, start_y),
                       arrowprops=dict(arrowstyle='->', lw=2, color='darkblue'))
        
        # 그래프 설정
        ax.set_xlim(0, 12)
        ax.set_ylim(2, 6)
        ax.set_aspect('equal')
        ax.axis('off')
        ax.set_title('LangGraph 코드 분석 워크플로우', fontsize=16, fontweight='bold', pad=20)
        
        # 범례 추가
        legend_elements = [
            mpatches.Circle((0, 0), 0.1, facecolor='lightgreen', edgecolor='black', label='시작'),
            mpatches.Rectangle((0, 0), 0.2, 0.1, facecolor='lightblue', edgecolor='black', label='분석 노드'),
            mpatches.Circle((0, 0), 0.1, facecolor='lightcoral', edgecolor='black', label='종료')
        ]
        ax.legend(handles=legend_elements, loc='upper right', bbox_to_anchor=(0.98, 0.98))
        
        plt.tight_layout()
        plt.show()
        
        # 워크플로우 설명
        print("\n🔄 LangGraph 워크플로우 설명:")
        print("1. START → style_check: 코드 스타일 분석 (PEP 8, 네이밍 등)")
        print("2. style_check → bug_check: 버그 가능성 분석 (구문오류, 런타임 에러)")
        print("3. bug_check → optimization_check: 최적화 분석 (성능 개선 제안)")
        print("4. optimization_check → improve_code: 분석 결과 반영하여 코드 개선")
        print("5. improve_code → END: 개선된 코드와 분석 보고서 반환")
    
    def show_graph_structure(self):
        """그래프 구조를 텍스트로 출력"""
        print("=" * 50)
        print("🏗️  LangGraph 구조")
        print("=" * 50)
        print("노드:")
        print("  📝 style_check - 코드 스타일 분석")
        print("  🐛 bug_check - 버그 가능성 분석") 
        print("  ⚡ optimization_check - 최적화 분석")
        print("  🔧 improve_code - 코드 개선")
        print("\n연결:")
        print("  START → style_check → bug_check → optimization_check → improve_code → END")
        print("\n상태 관리:")
        print("  - original_code: 원본 코드")
        print("  - style_analysis: 스타일 분석 결과")
        print("  - bug_analysis: 버그 분석 결과")
        print("  - optimization_analysis: 최적화 분석 결과")
        print("  - improved_code: 개선된 코드")
        print("  - analysis_complete: 분석 완료 여부")
    
    def analyze_code(self, code: str) -> Dict[str, Any]:
        """코드 분석 및 개선 실행"""
        initial_state = {
            "original_code": code,
            "style_analysis": {},
            "bug_analysis": {},
            "optimization_analysis": {},
            "improved_code": "",
            "analysis_complete": False
        }
        
        # 그래프 실행
        result = self.graph.invoke(initial_state)
        
        return {
            'original_code': result["original_code"],
            'style_analysis': result["style_analysis"],
            'bug_analysis': result["bug_analysis"],
            'optimization_analysis': result["optimization_analysis"],
            'improved_code': result["improved_code"],
            'overall_score': (
                result["style_analysis"]['score'] + 
                result["bug_analysis"]['score'] + 
                result["optimization_analysis"]['score']
            ) / 3
        }
    
    def print_analysis_report(self, analysis_result: Dict[str, Any]):
        """분석 결과 출력"""
        print("=" * 60)
        print("코드 분석 및 개선 보고서")
        print("=" * 60)
        
        # 전체 점수
        print(f"\n전체 점수: {analysis_result['overall_score']:.1f}/100")
        
        # 스타일 분석
        style = analysis_result['style_analysis']
        print(f"\n📋 스타일 분석 (점수: {style['score']}/100):")
        if style['issues']:
            print("  문제점:")
            for issue in style['issues']:
                print(f"    - {issue}")
        if style['suggestions']:
            print("  제안사항:")
            for suggestion in style['suggestions']:
                print(f"    - {suggestion}")
        
        # 버그 분석
        bug = analysis_result['bug_analysis']
        print(f"\n🐛 버그 분석 (점수: {bug['score']}/100):")
        if bug['potential_bugs']:
            print("  잠재적 버그:")
            for bug_issue in bug['potential_bugs']:
                print(f"    - {bug_issue}")
        if bug['warnings']:
            print("  경고사항:")
            for warning in bug['warnings']:
                print(f"    - {warning}")
        
        # 최적화 분석
        opt = analysis_result['optimization_analysis']
        print(f"\n⚡ 최적화 분석 (점수: {opt['score']}/100):")
        if opt['optimizations']:
            print("  최적화 제안:")
            for optimization in opt['optimizations']:
                print(f"    - {optimization}")
        if opt['performance_tips']:
            print("  성능 팁:")
            for tip in opt['performance_tips']:
                print(f"    - {tip}")
        
        print("\n" + "=" * 60)
        print("개선된 코드:")
        print("=" * 60)
        print(analysis_result['improved_code'])


# 사용 예제
def main():
    # 코드 분석기 생성
    analyzer = CodeAnalyzer()
    
    # 그래프 구조 시각화
    print("LangGraph 워크플로우를 시각화합니다...")
    analyzer.show_graph_structure()
    
    if VISUALIZATION_AVAILABLE:
        analyzer.visualize_graph()
    
    # 분석할 샘플 코드
    sample_code = '''
import os,sys
def badFunction(x,y):
	result=x+y
	file=open("test.txt","r")
	data=file.read()
	file.close()
	for i in range(len(data)):
		print(data[i])
	try:
		value=10/0
	except:
		pass
	return result
class myClass:
	def __init__(self):
		self.value=0
'''
    
    print("\n" + "="*60)
    print("코드 분석 시작...")
    print("="*60)
    
    # 코드 분석 실행
    result = analyzer.analyze_code(sample_code)
    analyzer.print_analysis_report(result)

if __name__ == "__main__":
    main()